# MAESTRIA INTELIGENCIA ARTIFICIAL


---



# LABORATORIO 6

INTEGRANTES:

- Edwin Montenegro
- Galo Travez

El objetivo de este taller es construir un modelo Transformer de traducción automática de español a inglés, utilizando PyTorch Lightning para estructurar el código de manera modular y eficiente. El taller también explora el uso de modelos preentrenados, como el MarianMTModel de Hugging Face, para comparar la creación de modelos de traducción desde cero frente a la utilización de modelos preentrenados.

# Cargar el dataset desde el archivo .txt

In [25]:
import pandas as pd
from transformers import BertTokenizer

# Cargar el dataset desde el archivo .txt
def load_dataset(filepath):
    with open(filepath, 'r', encoding='utf-8') as file:
        lines = file.readlines()

    # Procesar cada línea para obtener las frases en inglés y español
    data = []
    for line in lines:
        parts = line.strip().split('\t')
        if len(parts) >= 2:
            eng_sentence = parts[0]  # Frase en inglés
            spa_sentence = parts[1]  # Frase en español
            data.append((eng_sentence, spa_sentence))

    return pd.DataFrame(data, columns=["english", "spanish"])

# Cargar el dataset
dataset = load_dataset('/home/emontenegrob/Labs_NLP/data/spa.txt')

# Mostrar las primeras filas del dataset
print(dataset.head())

  english  spanish
0     Go.      Ve.
1     Go.    Vete.
2     Go.    Vaya.
3     Go.  Váyase.
4     Hi.    Hola.


# Cargar tokenizadores

In [27]:
# Cargar tokenizadores
tokenizer_src = BertTokenizer.from_pretrained("bert-base-uncased")  # Para inglés
tokenizer_tgt = BertTokenizer.from_pretrained("dccuchile/bert-base-spanish-wwm-uncased")  # Para español

# Tokenizar las frases
def tokenize_sentences(tokenizer, sentences, max_length):
    return tokenizer(sentences, padding=True, truncation=True, max_length=max_length, return_tensors="pt")

max_seq_length = 128
english_tokenized = tokenize_sentences(tokenizer_src, dataset["english"].tolist(), max_seq_length)
spanish_tokenized = tokenize_sentences(tokenizer_tgt, dataset["spanish"].tolist(), max_seq_length)

# MODELO TRANSFORMER

En nuestro código, usamos la capa `nn.Transformer`, que internamente usa multi-head attention. Esto permite que el modelo se enfoque en diferentes partes de la secuencia de entrada al mismo tiempo, lo cual es crucial para la traducción

Positional Encoding:

En el tutorial, se explica la importancia de las codificaciones posicionales, ya que los Transformers no tienen un orden secuencial por naturaleza. Hemos implementado esto con un tensor que contiene la información posicional:
`self.positional_encoding = nn.Parameter(torch.zeros(1, 1000, d_model))  # Positional Encoding`

In [31]:
import torch
import torch.nn as nn
import pytorch_lightning as pl
import torch.optim as optim

class TranslationTransformer(pl.LightningModule):
    def __init__(self, src_vocab_size, tgt_vocab_size, d_model, num_heads, num_encoder_layers, num_decoder_layers, dim_feedforward, dropout):
        super(TranslationTransformer, self).__init__()
        self.d_model = d_model
        self.src_embedding = nn.Embedding(src_vocab_size, d_model)
        self.tgt_embedding = nn.Embedding(tgt_vocab_size, d_model)
        self.positional_encoding = nn.Parameter(torch.zeros(1, 1000, d_model))  # Positional Encoding

        # Configurar batch_first=True para el Transformer
        self.transformer = nn.Transformer(d_model=d_model, nhead=num_heads, num_encoder_layers=num_encoder_layers,
                                          num_decoder_layers=num_decoder_layers, dim_feedforward=dim_feedforward, dropout=dropout, batch_first=True)
        self.fc_out = nn.Linear(d_model, tgt_vocab_size)

    def forward(self, src, tgt, src_mask=None, tgt_mask=None):
        # Mover src y tgt al mismo dispositivo del modelo
        src = src.to(self.device)
        tgt = tgt.to(self.device)

        # Mover máscaras al dispositivo adecuado si existen
        if src_mask is not None:
            src_mask = src_mask.to(self.device)
        if tgt_mask is not None:
            tgt_mask = tgt_mask.to(self.device)

        # src y tgt son [batch_size, seq_length]
        src = self.src_embedding(src) * torch.sqrt(torch.tensor(self.d_model, dtype=torch.float32)).to(self.device)
        tgt = self.tgt_embedding(tgt) * torch.sqrt(torch.tensor(self.d_model, dtype=torch.float32)).to(self.device)

        # Añadir la codificación posicional
        src = src + self.positional_encoding[:, :src.size(1), :].to(self.device)
        tgt = tgt + self.positional_encoding[:, :tgt.size(1), :].to(self.device)

        # Pasar por el Transformer
        output = self.transformer(src, tgt, src_mask, tgt_mask)

        # Proyección a vocabulario de destino
        output = self.fc_out(output)
        return output

    def configure_optimizers(self):
        optimizer = optim.Adam(self.parameters(), lr=0.0001)
        return optimizer

    def training_step(self, batch, batch_idx):
        src, tgt_input, tgt_output = batch

        # Crear máscaras para evitar que el modelo vea el futuro (auto-regresivo)
        src_mask = self.generate_square_subsequent_mask(src.size(1))
        tgt_mask = self.generate_square_subsequent_mask(tgt_input.size(1))

        # Asegurarse de que las máscaras también estén en la GPU
        src_mask = src_mask.to(self.device)
        tgt_mask = tgt_mask.to(self.device)

        # Obtener las predicciones del Transformer
        outputs = self(src, tgt_input, src_mask, tgt_mask)
        loss = nn.CrossEntropyLoss()(outputs.view(-1, outputs.size(-1)), tgt_output.view(-1))
        self.log('train_loss', loss)
        return loss

    def generate_square_subsequent_mask(self, size):
        """Genera una máscara para evitar que el Transformer vea el futuro en la secuencia"""
        mask = torch.triu(torch.ones(size, size) * float('-inf'), diagonal=1)
        return mask

    def validation_step(self, batch, batch_idx):
        src, tgt_input, tgt_output = batch
        src_mask = self.generate_square_subsequent_mask(src.size(1)).to(self.device)
        tgt_mask = self.generate_square_subsequent_mask(tgt_input.size(1)).to(self.device)
        outputs = self(src, tgt_input, src_mask, tgt_mask)
        loss = nn.CrossEntropyLoss()(outputs.view(-1, outputs.size(-1)), tgt_output.view(-1))
        self.log('val_loss', loss)
        return loss


In [32]:
from torch.utils.data import Dataset, DataLoader

class TranslationDataset(Dataset):
    def __init__(self, src_tokenized, tgt_tokenized):
        self.src_tokenized = src_tokenized
        self.tgt_tokenized = tgt_tokenized

    def __len__(self):
        return len(self.src_tokenized["input_ids"])

    def __getitem__(self, idx):
        src = self.src_tokenized["input_ids"][idx]
        tgt_input = self.tgt_tokenized["input_ids"][idx][:-1]  # Entrada al decodificador
        tgt_output = self.tgt_tokenized["input_ids"][idx][1:]  # Salida esperada del decodificador
        return src, tgt_input, tgt_output

class TranslationDataModule(pl.LightningDataModule):
    def __init__(self, english_tokenized, spanish_tokenized, batch_size):
        super().__init__()
        self.batch_size = batch_size
        self.train_dataset = TranslationDataset(english_tokenized, spanish_tokenized)

    def train_dataloader(self):
        return DataLoader(self.train_dataset, batch_size=self.batch_size, shuffle=True)

    def val_dataloader(self):
        return DataLoader(self.train_dataset, batch_size=self.batch_size)



In [35]:
from pytorch_lightning.callbacks import ModelCheckpoint

# Configurar hiperparámetros
src_vocab_size = tokenizer_src.vocab_size
tgt_vocab_size = tokenizer_tgt.vocab_size
d_model = 512
num_heads = 8
num_encoder_layers = 6
num_decoder_layers = 6
dim_feedforward = 2048
dropout = 0.1

# Inicializar el modelo
model = TranslationTransformer(src_vocab_size, tgt_vocab_size, d_model, num_heads, num_encoder_layers, num_decoder_layers, dim_feedforward, dropout)

# Inicializar el DataModule
batch_size = 32
data_module = TranslationDataModule(english_tokenized, spanish_tokenized, batch_size)

checkpoint_callback = ModelCheckpoint(
    dirpath='/content/checkpoints',  # Aquí especificas el directorio para guardar los checkpoints
    filename='best-checkpoint',
    save_top_k=1,  # Guardar el mejor modelo
    monitor='val_loss',  # Basado en la pérdida de validación
    mode='min'
)

trainer = pl.Trainer(
    max_epochs=2,
    callbacks=[checkpoint_callback],
    accelerator="gpu",
    devices=1
)

# Entrenar el modelo
trainer.fit(model, data_module)



INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
/usr/local/lib/python3.10/dist-packages/pytorch_lightning/callbacks/model_checkpoint.py:654: Checkpoint directory /content/checkpoints exists and is not empty.
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name          | Type        | Params | Mode 
------------------------------------------------------
0 | src_embedding | Embedding   | 15.6 M | train
1 | tgt_embedding | Embedding   | 15.9 M | train
2 | transformer   | Transformer | 44.1 M | train
3 | fc_out        | Linear      | 15.9 M | train
  | other params  | n/a         | 512 K  | n/a  
------------------------------------------------------
92.1 M    Trainable params
0         Non-trainable par

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=2` reached.


In [37]:
# (suponiendo que tienes un checkpoint guardado del modelo entrenado)
checkpoint_path = '/content/checkpoints/best-checkpoint.ckpt'

# Cargar el modelo desde el checkpoint, pasando los parámetros necesarios
model = TranslationTransformer.load_from_checkpoint(
    checkpoint_path=checkpoint_path,
    src_vocab_size=src_vocab_size,      # Tamaño del vocabulario de origen
    tgt_vocab_size=tgt_vocab_size,      # Tamaño del vocabulario de destino
    d_model=d_model,                    # Dimensión del modelo
    num_heads=num_heads,                # Número de cabezas de atención
    num_encoder_layers=num_encoder_layers,  # Capas del encoder
    num_decoder_layers=num_decoder_layers,  # Capas del decoder
    dim_feedforward=dim_feedforward,    # Dimensión del feedforward
    dropout=dropout                     # Tasa de dropout
)

# Mover el modelo a la GPU si está disponible
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)


In [45]:
import torch

# Función para traducir una frase en español a inglés
def translate_sentence(sentence, model, tokenizer_src, tokenizer_tgt, max_length=128):
    # Establecer el modelo en modo de evaluación
    model.eval()

    # Tokenizar la frase en español
    tokens = tokenizer_tgt.encode(sentence, return_tensors="pt", max_length=max_length, truncation=True, padding='max_length')

    # Mover los tokens a la GPU si es necesario
    tokens = tokens.to(model.device)

    # Crear un tensor vacío para los tokens de salida (frase en inglés)
    tgt_tokens = torch.zeros((1, max_length), dtype=torch.long).to(model.device)

    # Inicialmente, el token de inicio (CLS) será el primer token en la secuencia de destino
    tgt_tokens[0, 0] = tokenizer_src.cls_token_id

    # Generar la secuencia de salida paso a paso
    for i in range(1, max_length):
        # Crear una máscara para evitar mirar hacia adelante
        tgt_mask = model.generate_square_subsequent_mask(i).to(model.device)

        # Ejecutar el modelo para obtener los tokens predichos
        with torch.no_grad():
            outputs = model(src=tokens, tgt=tgt_tokens[:, :i], tgt_mask=tgt_mask)

        # Obtener el token con la mayor probabilidad
        next_token = outputs.argmax(dim=-1)[:, -1].item()

        # Agregar el siguiente token a la secuencia de salida
        tgt_tokens[0, i] = next_token

        # Si el token es el token de fin de secuencia (SEP), detener el bucle
        if next_token == tokenizer_src.sep_token_id:
            break

    # Decodificar los tokens de salida (frase en inglés)
    translated_sentence = tokenizer_src.decode(tgt_tokens[0], skip_special_tokens=True)
    return translated_sentence


In [46]:
# ======= TRADUCIR UNA FRASE =======

# Cargar los tokenizadores usados durante el entrenamiento
tokenizer_src = BertTokenizer.from_pretrained("bert-base-uncased")  # Para inglés (fuente)
tokenizer_tgt = BertTokenizer.from_pretrained("dccuchile/bert-base-spanish-wwm-uncased")  # Para español (destino)

# Frase de prueba en español
sentence_es = "¿Cómo estás?"

# Realizar la traducción
translated_sentence = translate_sentence(sentence_es, model, tokenizer_src, tokenizer_tgt)

# Mostrar la frase traducida
print(f"Frase en español: {sentence_es}")
print(f"Traducción al inglés: {translated_sentence}")

Frase en español: ¿Cómo estás?
Traducción al inglés: . \ 名 + e r λ 。 カ † 5 р r λ 。 カ † æ e r ¶ arrive ј _ \ エ 5 р r λ 。 \ エ _ \ エ 5 р. \ エ _ \ エ e r faced π + \ エ * [unused4] [unused0] [unused0] [unused0] [unused0] [unused0] [unused0] [unused0] [unused0] [unused0] [unused0] [unused0] [unused0] [unused0] [unused0] [unused0] [unused0] [unused0] [unused0] [unused0] [unused0] [unused0] [unused0] [unused0] [unused0] [unused0] [unused0] [unused0] [unused0] [unused0] [unused0] [unused0] [unused0] [unused0] [unused0] [unused0] [unused0] [unused0] [unused0] [unused0] [unused0] [unused0] [unused0] [unused0] [unused0] [unused0] [unused0] [unused0] [unused0] [unused0] [unused0] [unused0] [unused0] [unused0] [unused0] [unused0] [unused0] [unused0] [unused0] [unused0] [unused0] [unused0] [unused0] [unused0] [unused0] [unused0] [unused0] [unused0] [unused0] [unused0] [unused0] [unused0] [unused0] [unused0] [unused0]


# Explicación del problema en la traducción
El resultado de la traducción no está generando una secuencia coherente. Las razones pueden ser las siguientes:
- Es posible que el modelo no haya sido entrenado correctamente o que el número de épocas sea insuficiente para aprender la tarea de traducción.
- Podría haber errores en los datos o el proceso de entrenamiento, lo que puede hacer que el modelo no converja correctamente

# Conclusiones
- El modelo Transformer es poderoso, pero requiere un entrenamiento exhaustivo y un preprocesamiento cuidadoso de los datos.
- Entrenar un modelo Transformer desde cero para una tarea como la traducción automática puede ser difícil sin grandes cantidades de datos y tiempo de entrenamiento.
- Usar tokenizadores incorrectos o inconsistentes durante el entrenamiento e inferencia puede generar resultados incoherentes.

# MODELO PREENTRENADO TRANSFORMER


---



El modelo MarianMT de Hugging Face es un modelo preentrenado especializado en traducción automática. En este caso, hemos utilizado el modelo Helsinki-NLP/opus-mt-en-es, que está preentrenado para traducir de inglés a español

# Ventajas de modelos preentrenados:

- Los modelos como MarianMT han sido entrenados en grandes corpus multilingües y son capaces de generalizar bien a nuevas frases.
-  No es necesario entrenar el modelo desde cero, lo que ahorra tiempo y recursos computacionales.


In [49]:
from transformers import MarianTokenizer, MarianMTModel

# Modelo y tokenizador preentrenado para la traducción
# Vamos a usar el modelo MarianMT de Hugging Face
model_name = "Helsinki-NLP/opus-mt-en-es"  # Ejemplo de traducción de inglés a español
tokenizer = MarianTokenizer.from_pretrained(model_name)
translation_model = MarianMTModel.from_pretrained(model_name).to(device)

def translate_text(text, model, tokenizer):

    # Tokenizamos el texto
    encoded_text = tokenizer(text, return_tensors="pt", padding=True, truncation=True).to(device)

    # Generamos la traducción
    translated_tokens = model.generate(**encoded_text)

    # Decodificamos la salida y devolvemos el texto traducido
    translated_text = tokenizer.decode(translated_tokens[0], skip_special_tokens=True)
    return translated_text




# Ejemplo de uso:

In [50]:

input_sentence = "Most people have no idea of the giant capacity we can immediately command when we focus all of our resources on mastering a single area of our lives  "
translated_sentence = translate_text(input_sentence, translation_model, tokenizer)
print(f"Input sentence: {input_sentence}")
print(f"Translated sentence: {translated_sentence}")

Input sentence: Most people have no idea of the giant capacity we can immediately command when we focus all of our resources on mastering a single area of our lives  
Translated sentence: La mayoría de la gente no tiene idea de la capacidad gigante que podemos controlar inmediatamente cuando centramos todos nuestros recursos en dominar una sola área de nuestras vidas


In [51]:
# Ejemplo de uso:
input_sentence = "Keep your face always toward the sunshine and shadows will fall behind you  "
translated_sentence = translate_text(input_sentence, translation_model, tokenizer)
print(f"Input sentence: {input_sentence}")
print(f"Translated sentence: {translated_sentence}")

Input sentence: Keep your face always toward the sunshine and shadows will fall behind you  
Translated sentence: Mantén tu cara siempre hacia el sol y las sombras caerán detrás de ti


# CONCLUSIONES
MarianMT es un modelo Transformer preentrenado para tareas de traducción automática entre varios pares de idiomas. Fue entrenado usando grandes cantidades de datos paralelos (pares de frases traducidas) para aprender cómo transformar secuencias de texto de un idioma a otro. Está basado en la arquitectura del Transformer y se ha optimizado para realizar traducciones de manera eficiente y precisa. Utiliza técnicas de preentrenamiento masivo, lo que le permite generalizar bien en múltiples dominios y lenguajes

# Comparación con los modelos realizados en clase

---


Aunque los modelos Encoder-Decoder con RNNs fueron pioneros en la traducción automática, sus resultados pueden ser limitados en comparación con los modelos más recientes basados en Transformers. Los RNNs tienden a sufrir de problemas de captura de dependencias a largo plazo y mala generalización, lo que lleva a traducciones incorrectas o imprecisas.

Los modelos como MarianMT ofrecen una traducción más precisa y contextualmente adecuada. Son capaces de comprender mejor los pronombres posesivos, las expresiones temporales y otros aspectos gramaticales complejos, debido a su entrenamiento masivo y el uso de auto-atención.